In [1]:

import torch
from torch import nn
import os
import utils
import argparse
import json
import pickle as pkl
import traceback
from tqdm import tqdm

In [364]:
parser = argparse.ArgumentParser()
parser.add_argument('--model_file', type=str, default="data/hmm_params_1.pkl")
parser.add_argument('--test', action='store_true')
parser.add_argument('--train_file', type=str, default="data/train.bmes")
parser.add_argument('--test_file', type=str, default="")
parser.add_argument('--vocab_file', type=str, default="data/vocab.json")
parser.add_argument('--n_epochs', type=int, default=1)
parser.add_argument('--batch_size', type=int, default=64)
args = parser.parse_args([])
print(args)

# load vocab, tag2id
if not os.path.exists(args.vocab_file):
    utils.build_vocab(args.train_file, args.vocab_file)
vocab, tag2id = json.load(open(args.vocab_file))
rev_vocab = {v:k for k,v in vocab.items()}

Namespace(batch_size=64, model_file='data/hmm_params_1.pkl', n_epochs=1, test=False, test_file='', train_file='data/train.bmes', vocab_file='data/vocab.json')


In [365]:
len(vocab)

2928

In [382]:
import utils_hmm as utils
from importlib import reload
reload(utils)

X, Y, mask = utils.load_data(args.train_file, vocab, tag2id)



Loading data...


In [383]:

import utils
X, Y = utils.load_data(args.train_file, vocab, tag2id)

Loading data...


In [384]:
import pandas as pd
pcts = [0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
pd.Series(len(x) for x in X).describe(pcts)


count    20000.000000
mean        13.219500
std          9.714033
min          0.000000
25%          7.000000
50%         12.000000
75%         18.000000
90%         25.000000
95%         30.000000
99%         44.000000
max        165.000000
dtype: float64

In [5]:
from hmm import HMM
from hmm_batch import HMM as HMM1



In [6]:
hmm_ = HMM(vocab, tag2id, torch.device('cpu'))
hmm_1 = HMM1(vocab, tag2id, torch.device('cpu'))
hmm_, hmm_1

pi:
tensor([[0.4791],
        [0.0000],
        [0.0000],
        [0.5209]])
A:
tensor([[0.0000, 0.7008, 0.2992, 0.0000],
        [0.0000, 0.1247, 0.8753, 0.0000],
        [0.6694, 0.0000, 0.0000, 0.3306],
        [0.5853, 0.0000, 0.0000, 0.4147]])
log_pi:
tensor([[-7.3588e-01],
        [-1.0000e+03],
        [-1.0000e+03],
        [-6.5217e-01]])
log_A:
tensor([[-1.0000e+03, -3.5556e-01, -1.2066e+00, -1.0000e+03],
        [-1.0000e+03, -2.0816e+00, -1.3322e-01, -1.0000e+03],
        [-4.0139e-01, -1.0000e+03, -1.0000e+03, -1.1068e+00],
        [-5.3569e-01, -1.0000e+03, -1.0000e+03, -8.8012e-01]])
pi:
tensor([[0.4791],
        [0.0000],
        [0.0000],
        [0.5209]])
A:
tensor([[0.0000, 0.7008, 0.2992, 0.0000],
        [0.0000, 0.1247, 0.8753, 0.0000],
        [0.6694, 0.0000, 0.0000, 0.3306],
        [0.5853, 0.0000, 0.0000, 0.4147]])
log_pi:
tensor([[-7.3588e-01],
        [-1.0000e+03],
        [-1.0000e+03],
        [-6.5217e-01]])
log_A:
tensor([[-1.0000e+03, -3.5556e-01, -1

(HMM(), HMM())

In [360]:
fin = open('data/train.bmes')
vocab = {"<pad>": 0, "<unk>": 1}
# tag2id = {"<pad>": 0}
# vocab = {"<unk>": 0}
tag2id = {'B': 0, 'M': 1, 'E': 2, 'S': 3}

from collections import defaultdict
word_cnt = defaultdict(int)
for _, line in enumerate(fin):
    if line.strip() == "":
        continue
    ch, tag = line.strip().split()
    word_cnt[ch] += 1
    if tag not in tag2id:
        tag2id[tag] = len(tag2id)
        
df_cnt = pd.DataFrame([[k,v] for k,v in word_cnt.items()], columns=['word', 'cnt'])
df_cnt = df_cnt.sort_values(by='cnt', ascending=False).reset_index(drop=True)

for _,row in df_cnt.query('cnt>10').iterrows():
    vocab[row['word']] = len(vocab)
vocab

In [376]:
''.join(df_cnt.head(50)['word'].values.tolist()[:10])

'，的。、国一在中人了'

In [362]:
len(vocab)

2928

In [372]:
df_cnt.query('cnt>300').shape, df_cnt.query('cnt>300')['cnt'].sum() / df_cnt['cnt'].sum()


((943, 2), 0.912892127232749)

In [357]:
df_cnt.query('cnt>10')['cnt'].sum() / df_cnt['cnt'].sum(), df_cnt.query('cnt>10')['cnt'].sum()

(0.9964581526547703, 1819979)

In [9]:
torch.cat(hmm_.forward_alg([1,2,3,4]), dim=1)

tensor([[   -9.3063,   -18.0331,   -26.9254,   -36.9771],
        [-1007.8899,   -20.6349,   -27.2941,   -35.5722],
        [-1009.5573,   -19.6312,   -26.9258,   -35.1331],
        [   -8.5306,   -18.8868,   -27.6497,   -36.4039]])

In [278]:
import numpy as np
# x, mask = np.array([[1,2,3,4,5,6],[3,4,5,6,7,8]]), np.array([[1,1,1,1,1,0],[1,1,1,1,1,1]])
x, mask = np.array([[1,2,3,4,5,6]]), np.array([[1,1,1,1,1,0]])
alphas1 = hmm_1.forward_alg(x, mask)
betas1 = hmm_1.backward_alg(x, mask)

In [279]:
alphas = []
betas = []
gammas = []
xis = []

for r,obs in tqdm(enumerate(x), total=len(x)):
    L_obs = mask[r].sum()
    alpha = hmm_.forward_alg(obs[:L_obs])
    beta = hmm_.backward_alg(obs[:L_obs])

    alphas.append(alpha)
    betas.append(beta)

    gamma = []
    print('length:', L_obs)
    for t in range(L_obs):
        gamma.append(alpha[t] + beta[t] - torch.logsumexp(alpha[t] + beta[t], dim=0))
        # print("gamma:", alpha[t].shape, beta[t].shape, torch.logsumexp(alpha[t] + beta[t], dim=0).shape)
    gammas.append(gamma)

    xi = []
    for t in range(L_obs-1):
        xi_t = alpha[t] + hmm_.log_A + beta[t+1].T + hmm_.log_B[:,obs[t+1]].view(1,-1)
        denom = torch.logsumexp(xi_t, dim=[0,1])
        xi.append(xi_t - denom)
    xis.append(xi)

100%|██████████| 1/1 [00:00<00:00, 594.52it/s]

length: 5


## check alphas

In [280]:
torch.cat([torch.cat([a for a in alpha], dim=1).unsqueeze(0) for alpha in alphas], dim=0).shape, alphas1.shape

(torch.Size([1, 4, 5]), torch.Size([1, 4, 6]))

In [281]:
torch.cat([torch.cat(alpha, dim=1).unsqueeze(0) for alpha in alphas], dim=0)[0]

tensor([[   -9.3063,   -18.0331,   -26.9254,   -36.9771,   -44.3788],
        [-1007.8899,   -20.6349,   -27.2941,   -35.5722,   -44.6588],
        [-1009.5573,   -19.6312,   -26.9258,   -35.1331,   -43.6110],
        [   -8.5306,   -18.8868,   -27.6497,   -36.4039,   -43.9621]])

In [283]:
alphas1[0]

tensor([[   -9.3063,   -18.0331,   -26.9254,   -36.9771,   -44.3788,   -44.3788],
        [-1007.8899,   -20.6349,   -27.2941,   -35.5722,   -44.6588,   -44.6588],
        [-1009.5573,   -19.6312,   -26.9258,   -35.1331,   -43.6110,   -43.6110],
        [   -8.5306,   -18.8868,   -27.6497,   -36.4039,   -43.9621,   -43.9621]])

## check betas

In [284]:
torch.cat([torch.cat([a for a in beta], dim=1).unsqueeze(0) for beta in betas], dim=0).shape, betas1.shape

(torch.Size([1, 4, 5]), torch.Size([1, 4, 6]))

In [285]:
torch.cat([torch.cat([a for a in beta], dim=1).unsqueeze(0) for beta in betas], dim=0)[0], betas1[0]

(tensor([[-34.8742, -25.5698, -16.4639,  -7.9045,   0.0000],
         [-33.9336, -25.5679, -16.6523,  -7.9711,   0.0000],
         [-34.4353, -24.6913, -17.6851,  -8.5908,   0.0000],
         [-34.4111, -24.7810, -17.6356,  -8.4992,   0.0000]]),
 tensor([[-34.8742, -25.5698, -16.4639,  -7.9045,   0.0000,   0.0000],
         [-33.9336, -25.5679, -16.6523,  -7.9711,   0.0000,   0.0000],
         [-34.4353, -24.6913, -17.6851,  -8.5908,   0.0000,   0.0000],
         [-34.4111, -24.7810, -17.6356,  -8.4992,   0.0000,   0.0000]]))

## check_gammas

In [328]:
gammas1 = alphas1 + betas1
gammas1 = gammas1 - torch.logsumexp(gammas1, dim=1, keepdim=True)
gammas1[0]



tensor([[-1.4933e+00, -9.1563e-01, -7.0202e-01, -2.1943e+00, -1.6915e+00,
         -1.6915e+00],
        [-9.9914e+02, -3.5155e+00, -1.2591e+00, -8.5604e-01, -1.9715e+00,
         -1.9715e+00],
        [-1.0013e+03, -1.6352e+00, -1.9236e+00, -1.0366e+00, -9.2375e-01,
         -9.2375e-01],
        [-2.5443e-01, -9.8056e-01, -2.5980e+00, -2.2158e+00, -1.2748e+00,
         -1.2748e+00]])

In [289]:
torch.cat([torch.cat(gamma, dim=1).unsqueeze(0) for gamma in gammas], dim=0)

tensor([[[-1.4933e+00, -9.1563e-01, -7.0202e-01, -2.1943e+00, -1.6915e+00],
         [-9.9914e+02, -3.5155e+00, -1.2591e+00, -8.5604e-01, -1.9715e+00],
         [-1.0013e+03, -1.6352e+00, -1.9236e+00, -1.0366e+00, -9.2375e-01],
         [-2.5443e-01, -9.8056e-01, -2.5980e+00, -2.2158e+00, -1.2748e+00]]])

In [290]:
torch.cat([torch.cat(gamma, dim=1).unsqueeze(0) for gamma in gammas], dim=0).shape, gammas1.shape

(torch.Size([1, 4, 5]), torch.Size([1, 4, 6]))

## check xis1

In [291]:
xis1 = alphas1[:,:,:-1].transpose(1,2).unsqueeze(3) + hmm_1.log_A[None,None] + betas1[:,:,1:].transpose(1,2).unsqueeze(2) + hmm_1.log_B[:,x[:,1:]].transpose(0,1).transpose(1,2).unsqueeze(2)
xis1 = xis1 - torch.logsumexp(xis1, dim=[2,3], keepdim=True)
xis1.shape

torch.Size([1, 5, 4, 4])

In [292]:
torch.cat([torch.cat([tmp1.unsqueeze(0) for tmp1 in tmp], dim=0).unsqueeze(0) for tmp in xis], dim=0).shape

torch.Size([1, 4, 4, 4])

In [293]:
torch.cat([torch.cat([tmp1.unsqueeze(0) for tmp1 in tmp], dim=0).unsqueeze(0) for tmp in xis[:1]], dim=0)[0][0], xis1[0][0]


(tensor([[-1.0012e+03, -3.5155e+00, -1.6352e+00, -1.0009e+03],
         [-1.9997e+03, -1.0038e+03, -9.9915e+02, -1.9995e+03],
         [-1.0018e+03, -2.0034e+03, -2.0007e+03, -1.0022e+03],
         [-9.1563e-01, -1.0024e+03, -9.9965e+02, -9.8056e-01]]),
 tensor([[-1.0012e+03, -3.5155e+00, -1.6352e+00, -1.0009e+03],
         [-1.9997e+03, -1.0038e+03, -9.9915e+02, -1.9995e+03],
         [-1.0018e+03, -2.0034e+03, -2.0007e+03, -1.0022e+03],
         [-9.1563e-01, -1.0024e+03, -9.9965e+02, -9.8056e-01]]))

In [260]:
torch.cat([torch.cat([tmp1.unsqueeze(0) for tmp1 in tmp], dim=0).unsqueeze(0) for tmp in xis[:1]], dim=0).shape, xis1.shape

(torch.Size([1, 4, 4, 4]), torch.Size([2, 5, 4, 4]))

## check new_pi

In [295]:
new_pi1 = torch.exp(gammas1[:,:,0]).mean(dim=0, keepdim=True)
new_pi1

tensor([[0.2246, 0.0000, 0.0000, 0.7754]])

In [296]:
new_pi = torch.cat([torch.exp(g[0]) for g in gammas], dim=1).mean(dim=1, keepdim=True)
new_pi

tensor([[0.2246],
        [0.0000],
        [0.0000],
        [0.7754]])

## check newA

In [326]:
new_A = torch.zeros(hmm_.n_state, hmm_.n_state)#.to(device)
new_A_denom = torch.zeros(hmm_.n_state, 1)# .to(device)
for r,obs in enumerate(x):
    L_obs = mask[r].sum()
    for t in range(L_obs-1):
        new_A += torch.exp(xis[r][t])
        new_A_denom += torch.exp(gammas[r][t])
new_A = new_A / new_A_denom
new_A

tensor([[0.0000, 0.6242, 0.3758, 0.0000],
        [0.0000, 0.1473, 0.8527, 0.0000],
        [0.5760, 0.0000, 0.0000, 0.4240],
        [0.5929, 0.0000, 0.0000, 0.4071]])

In [327]:
newA = (torch.exp(xis1) * mask[:,1:,None,None]).sum(dim=[0,1])
newA_denom = (torch.exp(gammas1[:,:,:-1,None]).transpose(1,2) * mask[:,1:,None,None]).sum(dim=[0,1])
newA = newA / newA_denom
newA

tensor([[0.0000, 0.6242, 0.3758, 0.0000],
        [0.0000, 0.1473, 0.8527, 0.0000],
        [0.5760, 0.0000, 0.0000, 0.4240],
        [0.5929, 0.0000, 0.0000, 0.4071]], dtype=torch.float64)

In [321]:
mask[:,1:,None].shape, gammas1[:,:,:-1].transpose(1,2).shape

((1, 5, 1), torch.Size([1, 5, 4]))

In [274]:
torch.exp(xis1).sum(dim=[0,1]) / torch.exp(gammas1[:,:,:-1]).sum(dim=[0,2]).unsqueeze(1)

tensor([[0.0000, 1.5956, 0.6748, 0.0000],
        [0.0000, 0.4236, 2.1212, 0.0000],
        [1.3047, 0.0000, 0.0000, 0.6889],
        [0.8553, 0.0000, 0.0000, 0.6152]])

## check new_B

In [330]:
# update emission matrix
new_B = torch.zeros(hmm_.n_state, hmm_.n_vocab)#.to(device)
new_B_denom = torch.zeros(hmm_.n_state, 1)#.to(device)
for r,obs in enumerate(x):
    indicator = torch.arange(hmm_.n_vocab).unsqueeze(0).expand(hmm_.n_state, -1)#.to(hmm.device)
    L_obs = mask[r].sum()
    for t in range(L_obs):
        new_B += torch.exp(gammas[r][t]) * (indicator == obs[t]).float()
        new_B_denom += torch.exp(gammas[r][t])
new_B = new_B / new_B_denom
new_B

tensor([[0.0000, 0.1586, 0.2826,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0339,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1784,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4806, 0.2325,  ..., 0.0000, 0.0000, 0.0000]])

In [331]:
indicator = torch.LongTensor(x).unsqueeze(2) == torch.arange(hmm_1.n_vocab)
newB = (indicator.unsqueeze(1) * torch.exp(gammas1).unsqueeze(3)).sum(dim=[0,2]) / torch.exp(gammas1).sum(dim=[0,2]).unsqueeze(1)
newB
# indicator = torch.arange(hmm_1.n_vocab).unsqueeze(0).expand(hmm_1.n_state, -1).float()#.to(device)
# newB1 = torch.einsum('rtn,sn->sv', [torch.exp(gammas1.transpose(1,2)), (indicator == torch.LongTensor(x).unsqueeze(2).expand(-1, -1, hmm_.n_vocab).float())])
# newB1

tensor([[0.0000, 0.1404, 0.2501,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0292,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1308,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4096, 0.1982,  ..., 0.0000, 0.0000, 0.0000]])

In [334]:
gammas1.shape, mask.shape

(torch.Size([1, 4, 6]), (1, 6))

In [337]:
newB = torch.zeros(hmm_1.n_state, hmm_1.n_vocab)
mask_tensor = torch.tensor(mask).unsqueeze(1)

denom = (torch.exp(gammas1) * mask_tensor).sum(dim=[0,2]).unsqueeze(1)
for k in range(hmm_1.n_vocab):
    indicator = torch.LongTensor(x) == k # [bs, T]
    newB[:,k] = (indicator.unsqueeze(1) * torch.exp(gammas1) * mask_tensor).sum(dim=[0,2])
newB = newB / denom
newB

tensor([[0.0000, 0.1586, 0.2826,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0339,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1784,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4806, 0.2325,  ..., 0.0000, 0.0000, 0.0000]])

In [216]:
(indicator.unsqueeze(1) * torch.exp(gammas1).unsqueeze(3)).shape

torch.Size([2, 4, 6, 4700])

In [217]:
(indicator * torch.exp(gammas1[:,i,:]).unsqueeze(2)).shape

torch.Size([2, 6, 4700])

In [377]:
import numpy as np
def viterbi_decode(observed_sequence, states, start_prob, transition_prob, emission_prob):
    num_states = len(states)
    T = len(observed_sequence)
    
    # Initialize Viterbi matrix and backpointer matrix
    V = np.zeros((num_states, T))
    V[:, 0] = start_prob * emission_prob[:, observed_sequence[0]]
    backpointers = np.zeros((num_states, T), dtype=int)
    
    # Perform Viterbi algorithm
    for t in range(1, T):
        for s in range(num_states):
            temp = V[:, t-1] * transition_prob[:, s] * emission_prob[s, observed_sequence[t]]
            V[s, t] = np.max(temp)
            backpointers[s, t] = np.argmax(temp)
    
    # Decode the most likely state sequence
    best_path_prob = np.max(V[:, -1])
    best_path = [np.argmax(V[:, -1])]
    for t in range(T-1, 0, -1):
        best_path.insert(0, backpointers[best_path[0], t])
    
    return best_path, best_path_prob
# Example usage
observed_sequence = [0, 1, 0, 1, 0]
states = [0, 1]
start_prob = np.array([0.5, 0.5])
transition_prob = np.array([[0.6, 0.4],
                            [0.3, 0.7]])
emission_prob = np.array([[0.8, 0.2],
                          [0.2, 0.8]])
best_path, best_path_prob = viterbi_decode(observed_sequence, states, start_prob, transition_prob, emission_prob)
print("Best path:", best_path)
print("Best path probability:", best_path_prob)

Best path: [0, 1, 1, 1, 0]
Best path probability: 0.0024084480000000005
